In [1]:
# https://dplyr.tidyverse.org/reference/context.html
from datar.all import *

%run nb_helpers.py
nb_header(cur_group_id, cur_group_rows, cur_data, cur_data_all, cur_column, book='context')

### # cur_group_id  

##### gives a unique numeric identifier for the current group.


### # cur_group_rows  

##### gives the row indices for the current group.


### # cur_data  

##### gives the current data for the current group
(excluding grouping variables).  


### # cur_data_all  

##### gives the current data for the current group
(including grouping variables)  


### # cur_column  

##### Used in the functions of across. So we don't have to register it.


In [2]:
df = tibble(
  g=['a'] + ['b'] * 2 + ['c'] * 3,
  x=runif(6),
  y=runif(6)
)
gf = df >> group_by(f.g)

gf >> summarise(n = n())

,g,n
0,a,1
1,b,2
2,c,3


In [3]:
gf
# TODO: display groups

,g,x,y
0,a,0.975339,0.846483
1,b,0.206413,0.202191
2,b,0.152242,0.452938
3,c,0.775672,0.347262
4,c,0.450120,0.718517
5,c,0.281555,0.459833


In [4]:
gf >> mutate(id=cur_group_id()) 

,g,x,y,id
0,a,0.975339,0.846483,0
1,b,0.206413,0.202191,1
2,b,0.152242,0.452938,1
3,c,0.775672,0.347262,2
4,c,0.450120,0.718517,2
5,c,0.281555,0.459833,2


In [5]:
gf >> summarise(row=cur_group_rows()) 

[2021-04-16 17:51:54][datar][   INFO] `summarise()` has grouped output by ['g'] (override with `_groups` argument)


,g,row
0,a,0
1,b,1
2,b,2
3,c,3
4,c,4
5,c,5


In [6]:
gf_group = gf >> summarise(data=[cur_group()])
gf_group 

,g,data
0,a,g 0 a
1,b,g 1 b
2,c,g 2 c


In [7]:
gf_group >> pull(f.data, to='list')

[   g
 0  a,
    g
 1  b,
    g
 2  c]

In [8]:
gf_data = gf >> summarise(data=[cur_data()])
gf_data

,g,data
0,a,x y 0 0.975339 0.846483
1,b,x y 0 0.206413 0.202191 1 ...
2,c,x y 0 0.775672 0.347262 1 ...


In [9]:
gf_data >> pull(f.data, to='list')

[          x         y
 0  0.975339  0.846483,
           x         y
 0  0.206413  0.202191
 1  0.152242  0.452938,
           x         y
 0  0.775672  0.347262
 1  0.450120  0.718517
 2  0.281555  0.459833]

In [10]:
gf_data_all = gf >> summarise(data=[cur_data_all()])
gf_data_all

,g,data
0,a,g x y 0 a 0.975339 0.846483
1,b,g x y 0 b 0.206413 0.202...
2,c,g x y 0 c 0.775672 0.347...


In [11]:
gf_data_all >> pull(f.data, to='list')

[   g         x         y
 0  a  0.975339  0.846483,
    g         x         y
 0  b  0.206413  0.202191
 1  b  0.152242  0.452938,
    g         x         y
 0  c  0.775672  0.347262
 1  c  0.450120  0.718517
 2  c  0.281555  0.459833]

In [12]:
df >> select(f.x, f.y) >> mutate(
    across(
        everything(), 
        lambda x, cc: [cc + ' '] * x.shape[0] + (x**2).astype(str), cc=cur_column()
    )
)

[1, 2]


,x,y
0,x 0.9512852627845192,y 0.7165341775761459
1,x 0.042606297710791816,y 0.040881202819633165
2,x 0.023177695200093093,y 0.205152595878916
3,x 0.6016676558242666,y 0.12059120947029854
4,x 0.20260807452469115,y 0.5162664597846687
5,x 0.07927315418286633,y 0.21144628240897376


In [13]:
# or you can use x.name, since x is a Series
df >> mutate(across(
    [f.x, f.y], 
    lambda x: [x.name + ' '] * x.shape[0] + (x**2).astype(str)
))

,g,x,y
0,a,x 0.9512852627845192,y 0.7165341775761459
1,b,x 0.042606297710791816,y 0.040881202819633165
2,b,x 0.023177695200093093,y 0.205152595878916
3,c,x 0.6016676558242666,y 0.12059120947029854
4,c,x 0.20260807452469115,y 0.5162664597846687
5,c,x 0.07927315418286633,y 0.21144628240897376
